In [1]:
import torch
import torch.nn.functional as F
import random

## setup MLP

In [2]:
#setup globabl variables
words = open('names.txt','r').read().splitlines()
chars = sorted(list(set(''.join(words)))) #list of chars
stoi = {s:i+1 for i,s in enumerate(chars)}#mapping of chars to integers
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()} #map integers to chars
vocab_size = len(stoi)

In [3]:
#build the dataset out of blocks
block_size = 3

def build_dataset(words, block_size):
    '''input is a list of words and the size of each block
    output X maps block_size consecutive characters to the next character Y'''
    X, Y = [], []
    for word in words:
        context = [0]*block_size
        #print(word)
        for char in word + '.':
            ix = stoi[char]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] #crop and append
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

#80% of data will be used for training, 10% for hyperparam optimization, 10% for validation
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1], block_size) #training
Xdev, Ydev = build_dataset(words[n1:n2], block_size) #hyper-param dev
Xte, Yte = build_dataset(words[n2:], block_size) #testing

In [4]:
#set up parameters of MLP
n_embed = 10 #dimensions in character embeddings
n_hidden = 200 #number of neurons in the hidden MLP layer

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator = g)

#build a 2 layer neural net to predict Ys from Xs
#first layer goes to n_hidden
# we should multiply to change standard deviation of backprop weights
W1 = torch.randn((n_embed*block_size, n_hidden), generator=g)*0.01 #weights

#turn MLP layer to to 27 digits
W2 = torch.randn((n_hidden, vocab_size), generator=g)*0.01 #weights
b2 = torch.randn(vocab_size, generator=g)*0.01 #biases, multiple by 

#batch normalization
bngain = torch.ones((1,n_hidden))
bnbias = torch.zeros((1,n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

In [5]:
#training
max_steps = 10000
batch_size = 32

for i in range(max_steps):
    #construct minibatch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] #batches
    
    #forward pass
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1) #concatenate the vectors so you can do matrix multip
    pre_activations = embcat@W1
    
    #Batch normalization: want activations in hidden layer to be normal in the beginning, but then free to change!
    #use mean and standard deviation of the whole system, not just the batch, so we keep a running tally
    bmean = pre_activations.mean(0, keepdim=True)
    bstd = pre_activations.std(0, keepdim=True)
    pre_activations = (bngain*((pre_activations - bmean)/bstd)) + bnbias
    
    first_activations = torch.tanh(pre_activations)
    logits = first_activations@W2 + b2 #output
    loss = F.cross_entropy(logits, Yb) #compute loss 
    
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    #update
    lr = 0.1 #if i < 100000 else 0.01  #change learning rate as you surpass a certain level on training
    for p in parameters:
        p.data += -lr*p.grad 

In [6]:
#loss on testing data
#forward pass
training_emb = C[Xte]
training_embcat = training_emb.view(training_emb.shape[0], -1) #concatenate the vectors so you can do matrix multip
training_first_activations = torch.tanh(training_embcat@W1)
training_logits = training_first_activations@W2 + b2 #output
training_loss = F.cross_entropy(training_logits, Yte) #compute loss
training_loss.data

tensor(2.6228)

In [7]:
loss.data

tensor(2.5510)